# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] 1451599287 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1451599287 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to zg33@columbia.edu and will expire on October 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-10544 - Server binary: /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451599287.log
[INFO] GraphLab Server Version: 1.7.1


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_numpy_data()` from the second notebook of Week 2.

In [4]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [5]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    
    # select the columns of data_SFrame given by the features list 
    # into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]

    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [6]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns 
    # and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [7]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [8]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [10]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


In [12]:
print(np.linalg.norm(X/norms, axis=0))

[ 1.  1.  1.]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [24]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return (normalized_features, norms)

To test the function, run the following:

In [16]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [17]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [18]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [19]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [20]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [27]:
def compute_ro(normalized_features, output, weights, i):
    prediction = predict_output(normalized_features, weights)
    errors = output - prediction
    feature_i = normalized_features[:,i]
    rho_i = feature_i.dot(errors + weights[i]*feature_i)
    return rho_i

In [40]:
rhos = []
for i in range(3):
    rho_i = compute_ro(normalized_features, output, weights, i)
    rhos.append(rho_i)
rhos

[79400300.034929201, 87939470.772991076, 80966698.675965667]

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

Recall that we set $w_j = 0$ iff $|\rho_j| \leq \lambda/2$.

I.e. iff
$$ \lambda \geq 2|\rho_j| $$

In the case that $\rho_j > 0$ (our cases) this becomes:

$$\lambda \geq 2\rho_j $$

In our case if $\rho_1 = 87939470.772991076$ and $\rho_2 = 80966698.67596566$ which means...



In [32]:
Q5_1_lowerbound = 2*80966698.675965667
Q5_1_upperbound = 2*87939470.772991076
print (Q5_1_lowerbound)
print(Q5_1_upperbound)

161933397.352
175878941.546


...that we satisfy the condition for 

$$ 161933397.352 \leq \lambda < 175878941.546 $$

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [33]:
Q5_2_lowerbound = Q5_1_upperbound
Q5_2_lowerbound

175878941.54598215

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [57]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights,
                                  l1_penalty):
    """
    Note: feature_matrix should be normalized.
    """
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = compute_ro(feature_matrix, output, weights, i)

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [49]:
# should print 0.425558846691
import math
features = np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                                                 [2./math.sqrt(13),3./math.sqrt(10)]])
print(features)
print(normalize_features(features))
print lasso_coordinate_descent_step(1, features, 
                                        np.array([1., 1.]), 
                                        np.array([1., 4.]), 0.1)

[[ 0.83205029  0.31622777]
 [ 0.5547002   0.9486833 ]]
(array([[ 0.83205029,  0.31622777],
       [ 0.5547002 ,  0.9486833 ]]), array([ 1.,  1.]))
l1_penalty = 0.100000
0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [63]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, 
                                      tolerance):
    max_weights_change = float('inf')
    weights = initial_weights
    iteration = 1
    while max_weights_change > tolerance:
        max_weights_change = 0.0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            max_weights_change = max(max_weights_change, np.abs(weights[i] - old_weights_i))
        print("step %d. Max weight change: %f" % (iteration, max_weights_change))
        iteration += 1
    return weights

In [64]:
5 < float('inf')
np.abs(-4.3)

4.2999999999999998

Using the following parameters, learn the weights on the sales dataset. 

In [65]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [66]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [67]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

step 1. Max weight change: 79400304.658051
step 2. Max weight change: 9138168.366497
step 3. Max weight change: 8194809.409370
step 4. Max weight change: 6598905.038273
step 5. Max weight change: 5522173.194295
step 6. Max weight change: 4621129.810313
step 7. Max weight change: 3867108.106248
step 8. Max weight change: 3236118.810607
step 9. Max weight change: 2708086.939551
step 10. Max weight change: 2266213.109399
step 11. Max weight change: 1896439.062647
step 12. Max weight change: 1587000.403192
step 13. Max weight change: 1328052.310954
step 14. Max weight change: 1111356.328002
step 15. Max weight change: 930018.251241
step 16. Max weight change: 778268.792690
step 17. Max weight change: 651280.029038
step 18. Max weight change: 545011.800818
step 19. Max weight change: 456083.174344
step 20. Max weight change: 381664.876995
step 21. Max weight change: 319389.283634
step 22. Max weight change: 267275.090397
step 23. Max weight change: 223664.279321
step 24. Max weight change: 

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

In [71]:
y_predict = predict_output(normalized_simple_feature_matrix, weights)
errors = output - y_predict
RSS = errors.dot(errors)
Q5_3_RSS = RSS
Q5_3_weights = weights
Q5_3_nonzeros = [ "constant", "sqft_living"]
Q5_3_weights

array([ 21624998.36636291,  63157246.78545421,         0.        ])

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [72]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [73]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [75]:
(train_feature_matrix, train_output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, norm_training) = normalize_features(train_feature_matrix)

In [79]:
norm_training

array([  1.31848398e+02,   4.60040216e+02,   2.96850552e+02,
         2.99962419e+05,   5.81709718e+06,   2.04107815e+02,
         1.15325626e+01,   1.05933942e+02,   4.57793622e+02,
         1.02101959e+03,   2.59726472e+05,   7.01224951e+04,
         2.59922094e+05,   5.36953839e+04])

In [84]:
c = train_feature_matrix[:,0]
np.sqrt(len(c))

131.84839779079607

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [86]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, train_output, np.zeros(14), 
                                              1e7, 1.0)

step 1. Max weight change: 71114625.752809
step 2. Max weight change: 5024356.368690
step 3. Max weight change: 5012324.913953
step 4. Max weight change: 5000676.420862
step 5. Max weight change: 4991243.173140
step 6. Max weight change: 4984361.343354
step 7. Max weight change: 4979504.929205
step 8. Max weight change: 4973806.565255
step 9. Max weight change: 2879222.999384
step 10. Max weight change: 2706150.766220
step 11. Max weight change: 2532295.648642
step 12. Max weight change: 2368316.577822
step 13. Max weight change: 2213124.350666
step 14. Max weight change: 1552443.600131
step 15. Max weight change: 507838.033159
step 16. Max weight change: 438814.370591
step 17. Max weight change: 371479.622608
step 18. Max weight change: 313415.105581
step 19. Max weight change: 264244.832139
step 20. Max weight change: 222753.860800
step 21. Max weight change: 187770.667306
step 22. Max weight change: 158280.093357
step 23. Max weight change: 133420.901606
step 24. Max weight change: 

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [91]:
weights1e7
Q5_4_nonzeros = [ "constant" ]
for i in range(1,14):
    if weights1e7[i] != 0.0:
        print i
        Q5_4_nonzeros.append(all_features[i-1])
Q5_4_nonzeros

3
6
7


['constant', 'sqft_living', 'waterfront', 'view']

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [92]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, train_output, np.zeros(14), 
                                              1e8, 1.0)

step 1. Max weight change: 71114625.752809
step 2. Max weight change: 0.000000


In [93]:
Q5_5_nonzeros = [ "constant" ]
for i in range(1,14):
    if weights1e8[i] != 0.0:
        print i
        Q5_5_nonzeros.append(all_features[i-1])
Q5_5_nonzeros

['constant']

***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [94]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, train_output, np.zeros(14), 
                                              1e4, 5e5)

step 1. Max weight change: 71114625.752809
step 2. Max weight change: 8464015.947382
step 3. Max weight change: 6818451.358693
step 4. Max weight change: 5570165.219010
step 5. Max weight change: 4559838.467594
step 6. Max weight change: 3732056.697149
step 7. Max weight change: 3053211.613526
step 8. Max weight change: 2497517.435575
step 9. Max weight change: 2043868.973246
step 10. Max weight change: 1674717.709057
step 11. Max weight change: 1375420.756922
step 12. Max weight change: 1296621.814296
step 13. Max weight change: 1257340.936098
step 14. Max weight change: 1228271.248190
step 15. Max weight change: 1206854.364715
step 16. Max weight change: 1191059.405980
step 17. Max weight change: 1179030.770175
step 18. Max weight change: 1169777.433448
step 19. Max weight change: 1162340.675886
step 20. Max weight change: 1155980.529008
step 21. Max weight change: 1150139.679659
step 22. Max weight change: 1144406.327518
step 23. Max weight change: 1138483.918053
step 24. Max weight

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [95]:
Q5_6_nonzeros = [ "constant" ]
for i in range(1,14):
    if weights1e4[i] != 0.0:
        print i
        Q5_6_nonzeros.append(all_features[i-1])
Q5_6_nonzeros

1
2
3
4
5
6
7
8
9
10
11
12
13


['constant',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [99]:
weights1e4_normalized = weights1e4 / norm_training
weights1e7_normalized = weights1e7 / norm_training
weights1e8_normalized = weights1e8 / norm_training
print weights1e7_normalized[3]

161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [100]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [101]:
y_predict = predict_output(test_feature_matrix, weights1e4_normalized)
errors = test_output - y_predict
RSS1e4 = errors.dot(errors)
RSS1e4

227781004760225.38

In [102]:
y_predict = predict_output(test_feature_matrix, weights1e7_normalized)
errors = test_output - y_predict
RSS1e7 = errors.dot(errors)
RSS1e7

275962079909185.38

In [103]:
y_predict = predict_output(test_feature_matrix, weights1e8_normalized)
errors = test_output - y_predict
RSS1e8 = errors.dot(errors)
RSS1e8

537166150034084.88

***QUIZ QUESTION***

Which model performed best on the test data?

In [104]:
Q5_7_best_model = "1e4"

In [106]:
print(Q5_1_lowerbound)

161933397.352


In [107]:
print(Q5_1_upperbound)

175878941.546


In [108]:
print(Q5_3_RSS)

1.63049248148e+15


In [109]:
print(Q5_3_nonzeros)

['constant', 'sqft_living']


In [110]:
print(Q5_4_nonzeros)

['constant', 'sqft_living', 'waterfront', 'view']


In [111]:
print(Q5_5_nonzeros)

['constant']


In [112]:
print(Q5_6_nonzeros)

['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [113]:
print(Q5_7_best_model)

1e4
